# **Introduction**

***

<p style="font-size:18px;">  This Case study contains the data from the book "Machine Learning in R" by Brett Lantz. Lantz notes that:
    
> <p style="font-size:18px;">The dataset is a simulated dataset containing medical expenses for patients in the United States. The data was created for the book using demographic statistics from the U.S. Census Bureau, and thus approximately reflects real-world conditions.
    
> <p style="font-size:18px;"> The insurance.csv file includes 1,338 examples of beneficiaries currently enrolled
in the insurance plan, with features indicating characteristics of the patient as well as the total medical expenses charged to the plan for the calendar year. The features are:
    
> <ul style="font-size:18px;"><li>age: This is an integer indicating the age of the primary beneficiary (excluding
> those above 64 years, since they are generally covered by the government).</li> 
> <li>sex: This is the policy holder's gender, either male or female.</li>
> <li>bmi: This is the body mass index (BMI), which provides a sense of how over
> or under-weight a person is relative to their height. BMI is equal to weight (in
> kilograms) divided by height (in meters) squared. An ideal BMI is within the
> range of 18.5 to 24.9.</li>
> <li>children: This is an integer indicating the number of children / dependents
> covered by the insurance plan.</li>
> <li>smoker: This is yes or no depending on whether the insured regularly
> smokes tobacco.</li>
> <li>region: This is the beneficiary's place of residence in the U.S., divided into
> four geographic regions: northeast, southeast, southwest, or northwest.
> Forecasting Numeric Data – Regression Methods</li></ul>

Lantz, B. (2013). Machine learning with R : learn how to use R to apply powerful machine learning methods and gain an insight into real-world applications. Packt Publishing.

<p style="font-size:18px;">  Throughout this case study I will be exploring medical data and how different factors impact the cost of a person's medical bills. I will begin by analyzing and cleaning the data before moving on to using machine learning algorithms to predict a person's medical bill based on several factors. 

##### **Importing libarys and data**

In [1]:
import numpy as np 
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

print("Libarys imported")

Libarys imported


In [2]:
df = pd.read_csv("/kaggle/input/insurance/insurance.csv")

print("Data imported")

df.head()

Data imported


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


# **Data Cleaning** 
    
***  

<p style="font-size:18px;"> Now that the data has been imported it is important to ensure that the data is clean and contains no null or incorrect values. I begin by getting info on the dataset.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


<p style="font-size:18px;">This tells me the number of values per column, the number of non null values and the data type per column.
<p style="font-size:18px;">  Fortunately in this circumstance the data does not contain any null values however there are 3 columns (sex, smoker and region) which have an object data type which will need to be changed when using ML algorithms. For the time being I will keep these columns with their current data as it will allow for my analysis to be more self explanatory. To explore this further I wanted to see the range of values per column.

In [4]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
age,1338.0,39.207025,14.049960,18.0000,27.00000,39.000,51.000000,64.00000
bmi,1338.0,30.663397,6.098187,15.9600,26.29625,30.400,34.693750,53.13000
children,1338.0,1.094918,1.205493,0.0000,0.00000,1.000,2.000000,5.00000
charges,1338.0,13270.422265,12110.011237,1121.8739,4740.28715,9382.033,16639.912515,63770.42801


<p style="font-size:18px;"> From this I can see that the minimum age in the dataset is 18 and the oldest is 64. From this it appears that all the columns have an accurate and appropriate range.

# **Data Exploration & Analysis** 
    
***
    

<p style="font-size:18px;"> To begin exploring the data, I wanted to get an overview of the data and how ages impact costs of medical bills. Please see the graph below. Most of the points follow a similar trend in which the older a person the higher their medical bills are. Although most of the data points are within a small range there are several points which are far out of the norm. To investigate this further I Initially had the points on the graph colored based on each person's sex (male or female) however this did not give me any further insight. As such I decided to color the points on the graph based on whether or not the person was a smoker. This then showed me that there appears to be a huge difference in medical bills which is dependent on whether a person is a smoker or not. 

In [5]:
# Scatter plot
fig = px.scatter(df, x='charges', y='age', color='smoker', title='Age vs Charges by Smokers', labels={'charges': 'Charges', 'age': 'Age', 'sex': 'Sex'})

fig.show()

<p style="font-size:18px;"> To explore this further I decided to plot the average medical charges by age for both smokers and non-smokers. Please see the graph below:

In [6]:
# Average charges by smoker status and age
avg_charges = df.groupby(['smoker', 'age'])['charges'].mean().reset_index()

# Line plot
fig = px.line(avg_charges, x='age', y='charges', color='smoker', title='Average Charges by Smoker and Age', 
              labels={'charges': 'Average Charges', 'age': 'Age', 'smoker': 'Smoker'})

# Update the y-axis range and adjust the figure height
fig.update_layout(yaxis_range=[0, df['charges'].max() * 1.1])

fig.show()

print("Average charge for smokers regardless of age $", round(df[df['smoker'] == 'yes']['charges'].mean(),2))
print("Average charge for non-smokers regardless of age $", round(df[df['smoker'] == 'no']['charges'].mean(),2))

Average charge for smokers regardless of age $ 32050.23
Average charge for non-smokers regardless of age $ 8434.27


<p style="font-size:18px;"> Wow! Smoking has a huge impact on a person's medical costs and this only increases by age. I decided to find the average medical bill for smokers and non-smoker regardless of age. On average, smokers' medical bills were 3.8x more expensive than non-smokers.
    
<p style="font-size:18px;"> Next I wanted to explore a different factor so I turned my attention to the number of children a person had and whether or not this could impact a person's medical bills. I was expecting to see that the more children a person had the more expensive their medical bills would be as I thought that a person with 5 children is likely to be older than a person with 1 child. I plotted this on the graph below:

In [7]:
# Scatter plot
fig = px.scatter(df, x='charges', y='children', color='age', title='Age by Children with Charges', 
                 labels={'charges': 'Charges', 'children': 'Children', 'age': 'Age'}, color_continuous_scale='Portland')


fig.show()

<p style="font-size:18px;"> The data shows that the more children a person has the higher the minimum charge tends to be but there doesn't appear to be enough data for users with more than 3 children to draw any accurate conclusions. In addition, age didn't play as much of a factor as I was expecting as there were several people with over 3 children who were 30 years old or less. As such I felt I should explore a different factor more specifically how BMI can affect medical bills, I expect to see the higher a person's BMI the higher their medical bills. Because I have previously found that smoking is a key factor on the medical charges I used the smoking column to color the points on the graph. 

In [8]:
# Scatter plot
fig = px.scatter(df, x='charges', y='bmi', color='smoker', title='Charges by BMI', labels={'charges': 'Charges', 'bmi': 'BMI', 'sex': 'Sex'})

fig.update_layout(yaxis_range=[0, df['bmi'].max() * 1.1], xaxis_range = [0,70000])

fig.show()

<p style="font-size:18px;"> To my surprise BMI did not play as much of a factor as I was expecting. For non-smokers there seems to be no correlation between BMI and medical charges. However there does appear to be a positive correlation between BMI and chargers for smokers. Smokers with a BMI over 30 tend to pay the most for their medical bills.
    
<p style="font-size:18px;"> Next, I wanted to visualize if the region a person was living in was correlated to the charges they were paying. I decided to visualize this as a box plot, once again split between smokers and non-smokers. 

In [9]:
# Box plot
fig = px.box(df, x='charges', y='region', color='smoker', 
             title='Charges by Region', 
             labels={'charges': 'Charges', 'region': 'Region', 'sex': 'Sex'})

# Update x-axis range
fig.update_layout(xaxis_range=[0, 70000])

fig.show()

<p style="font-size:18px;"> The data shows that there are minor differences between the charges and the region the person got treated. The biggest differences being that the south regions have a much higher median than the northern regions for smokers. However across the board there are minimal differences in the range of charges per region.
    
<p style="font-size:18px;"> One factor which I was yet to consider was whether the sex of a person would impact the cost of their medical bills. I decided to plot the sex of each person by age against the medial charges. I did not expect there to be any difference between sex and medical charges may be slightly higher in women due to pregnancies and associated costs. I plotted this graph below.

In [10]:
# Box Plot
fig = px.box(df, x='age', y='charges', color='sex', 
             title='Charges by Age', 
             labels={'charges': 'Charges', 'age': 'Age', 'sex': 'Sex'})

# Update the y-axis range
fig.update_layout(yaxis_range=[0, 70000])

fig.show()

<p style="font-size:18px;"> As seen previously as the age of a person increases their minimum medical charges increase. However this plot shows that there are some huge differences between the range of medical charges between men and women at different ages for example 23, 50 and 40 to name a few. It would be interesting to understand why this is the case unfortunately the data does not provide any information behind the medical bill charges as it could be that 23 is the most common age for pregnancy which is why the female medical expenses are significantly higher than men for this age. Without further data I unfortunately cannot draw accurate conclusions to why the data is showing several discrepancies between sex and charges. 

<p style="font-size:18px;"> In some cases there is a very small range of medical charges, could this be due to a lack of data for medical charges at this age? I decided to find out what percentage of the data is from men and women. Then determine the frequency of the data by age to see if it could be the case that there is a lack of data for several ages. Please see my plots below:

In [11]:
# PIE CHART SEX PERCENTAGE PLOT

# Count of male and female values
sex_counts = df['sex'].value_counts().reset_index()
sex_counts.columns = ['sex', 'count']

# Default Plotly colors
default_colors = px.colors.qualitative.Plotly

# Color swap for 'male' and 'female' to keep consistent
color_map = {'male': default_colors[1], 'female': default_colors[0]}

# Pie chart
fig = px.pie(sex_counts, values='count', names='sex', title='Number of Male vs Female Values',
             color='sex',color_discrete_map=color_map) 

fig.show()

#####################################################################################################################################

# BAR CHART AGE FREQUENCY PLOT

# Count of male and female values grouped by age
sex_age_counts = df.groupby(['age', 'sex']).size().reset_index(name='count')

# Bar Chart
fig = px.bar(sex_age_counts, x='age', y='count', color='sex', barmode='group',title='Number of Male vs Female Values by Age',
             labels={'count': 'Count', 'age': 'Age', 'sex': 'Sex'})

fig.update_layout(yaxis_range=[0, 37])

fig.show()

<p style="font-size:18px;"> From this I can see that the data is mostly uniform with men making up 50.5% of all the data and women making up the remaining 49.5%. However when looking at the distribution of data by age there is a massive discrepancy as the data is heavily weighted to people aged 18 and 19. The rest of the ages remain mostly even and flat with a slight decrease from 54. This distribution of data will need to be considered when I implement ML algorithms in the next stage.
    
<p style="font-size:18px;"> Next I felt it would be great to get a high level overview of the data as such I took the average charge by age and plotted this on a scatter graph and added a trend line to see the trend in the data.

In [12]:
# Average charge by age
average_charge_by_age = df.groupby('age')['charges'].mean().reset_index()

# line graph
fig = px.scatter(average_charge_by_age, x='age', y='charges', 
              title='Average Charge by Age', trendline='ols',
              labels={'age': 'Age', 'charges': 'Average Charge'})

fig.show()

<p style="font-size:18px;"> I feel that I have a good understanding of the data. The final thing I wanted to visualize before using machine learning algorithms is to see the correlation between all the columns in the data set. For this to work however I will need to convert and map the non-numerical columns to numerical numbers this is also crucial for the machine learning algorithms to work.

In [13]:
#Converts the values in the sex column from male and female to 1 and 0
df['sex'] = df['sex'].map({'male': 1, 'female': 0})

#Converts the values in the smoker column from yes and no to 1 and 0
df['smoker'] = df['smoker'].map({'yes': 1, 'no': 0})

#Converts the 4 diffrent regions to numerical values
region_mapping = {"southwest": 1,"southeast": 2,"northwest": 3,"northeast": 4}
df["region"] = df["region"].map(region_mapping)

df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,1,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.000,3,0,2,4449.46200
3,33,1,22.705,0,0,3,21984.47061
4,32,1,28.880,0,0,3,3866.85520


<p style="font-size:18px;"> I have converted the following columns to numerical values, the "sex" column now shows 1 for men and 0 for women. the "smoker" column now shows 1 if that person smokes and 0 if they do not smoke. Finally each of the regions has been mapped to a number, 1 for south west, 2 for south east, 3 for north west and 4 for north east. Now that the data set only contains numeric values I can now visualize the correlation between the columns.

In [14]:
# Heatmap
fig = ff.create_annotated_heatmap(z=df.corr().values, x=list(df.corr().columns),y=list(df.corr().index),
                                  colorscale='Blues', annotation_text=df.corr().round(2).values, showscale=True)

fig.update_layout(title="Correlation Heatmap",width=850, height=850)

fig.show()

<p style="font-size:18px;"> It is great to see how each of the columns are correlated to each other. The correlation confirms my previous findings that the cost of a medical bill is strongly linked to whether or not a person smokes and the age of a person. The rest of the columns appear to have minimal correlation on a person's medical charges. 
    
<p style="font-size:18px;"> I will conclude my exploratory data analysis here before looking at how different machine learning algorithms could predict a person's medical charges. From the data I have found that there are multiple factors which could impact a person's medical charges. Unfortunately the data set does not provide further detail such as the duration a person has been smoking for or the reasons for their medical charges as this could drastically impact the analysis and allow for greater insight. I hope that my analysis has provided useful insights to the data set and how medical charges can be impacted by other factors such as age, smoking status and BMI.

# **Spliting Data** 
    
***
    

<p style="font-size:18px;"> Before I can run any machine learning algorithms on the data I need to split the data into X and y. X being the features and y being the target which we are trying to predict. Once X and y are defined I then need to split the data into a training data set and a testing data set. In the code below I have allocated 0.2 or 20% of the data for testing/evaluating the models and 80% of the data for training the models. The data is then scaled as this helps the accuracy of the ML I will be using today.

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Defining features and target
X = df[["age", "sex", "bmi", "children", "smoker", "region"]]
y = df["charges"]

# Data is split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# **Linear Regression** 
    
***
    

<p style="font-size:18px;"> To begin I will start with linear regression. Linear regression attempts to model the relationship between the target and one or more features by fitting a linear equation to the observed data. It does this by minimizing the sum of the squared differences between the observed and predicted values. In this circumstance I will be doing multiple linear regression as I have multiple features. Firstly I import the required libraries from sklearn.

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import cross_val_score

<p style="font-size:18px;"> Now that the libraries have been imported I can create the linear regression model and fit the previously split training data to the model.

In [17]:
# Create and train the model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

LinearRegression()

<p style="font-size:18px;"> The model has now been trained as such I can now make predictions using the test data as this is data which the model has not seen before. To better visualize this I plotted the regression line against the predicted and actual values to see how accurate this model is. In addition I printed out the key metrics to determine the accuracy of the model.    

In [18]:
# Make predictions
y_pred = lr_model.predict(X_test)

In [19]:
# Residuals calculation
residuals = y_test - y_pred

#Scatter plot of actual vs predicted values
fig_actual_vs_pred = go.Figure()

fig_actual_vs_pred.add_trace(go.Scatter(x=y_test, y=y_pred, mode='markers', name='Predicted vs Actual',
                                        text=[f"Predicted: ${pred:.2f} <br>Actual: ${act:.2f} <br>Residual: ${res:.2f}" for pred, act, res in zip(y_pred, y_test, residuals)],
                                        hoverinfo='text'))

fig_actual_vs_pred.add_trace(go.Scatter(x=[y_test.min(), y_test.max()], y=[y_test.min(), y_test.max()], 
                                        mode='lines', name='Ideal Fit', line=dict(color='black', dash='dash')))

fig_actual_vs_pred.update_layout(title='Actual vs Predicted Values', xaxis_title='Actual', yaxis_title='Predicted')

fig_actual_vs_pred.show()

#Evaluating the model
print('R-squared:', r2_score(y_test, y_pred).round(5))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred).round(3))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)).round(3))
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred).round(3))
print('Coefficients:', lr_model.coef_.round(3))
print('Intercept:', lr_model.intercept_.round(3))

R-squared: 0.78335
Mean Squared Error: 33635210.431
Root Mean Squared Error: 5799.587
Mean Absolute Error: 4186.509
Coefficients: [ 3.616109e+03 -9.393000e+00  2.028309e+03  5.166630e+02  9.557143e+03
  3.023880e+02]
Intercept: 13346.09


<p style="font-size:18px;"> For a basic linear regression model the accuracy is okay. It's clear to see by looking at the graph that the model is much more accurate at the lower charges and becomes less accurate as the charges increase. To check if this was the best linear model I ran the data on ridge and lasso regression models to see if either of these models would be more accurate.  

In [20]:
from sklearn.linear_model import  Ridge, Lasso

# RIDGE MODEL ###############################################################################

# Create and train the model
rdg_model = Ridge()
rdg_model.fit(X_train, y_train)

# Make predictions
y_pred = rdg_model.predict(X_test)

#Evaluating the model
print('Ridge - R-squared:', r2_score(y_test, y_pred).round(5))
print('Ridge - Mean Absolute Error:', mean_absolute_error(y_test, y_pred).round(3))
print('Ridge - Mean Squared Error:', mean_squared_error(y_test, y_pred).round(3))
print("-------------------------------------------")

# LASSO MODEL ###############################################################################

# Create and train the model
lso_model = Lasso()
lso_model.fit(X_train, y_train)

# Make predictions
y_pred = lso_model.predict(X_test)

#Evaluating the model
print('Lasso - R-squared:', r2_score(y_test, y_pred).round(5))
print('Lasso - Mean Absolute Error:', mean_absolute_error(y_test, y_pred).round(3))
print('Lasso - Mean Squared Error:', mean_squared_error(y_test, y_pred).round(3))

Ridge - R-squared: 0.7833
Ridge - Mean Absolute Error: 4187.972
Ridge - Mean Squared Error: 33641818.589
-------------------------------------------
Lasso - R-squared: 0.78333
Lasso - Mean Absolute Error: 4186.624
Lasso - Mean Squared Error: 33637843.016


<p style="font-size:18px;"> It appears that all of the linear models have a very similar accuracy which is okay for a basic model. I believe the accuracy could likely be improved by using a different model.

# **Random Forest Regressor**
    
***

<p style="font-size:18px;"> Next I will try using a random forest which is an ensemble learning method primarily used for classification and regression tasks. It works by constructing a multitude of decision trees during training and outputting the class that is the mean prediction of the individual trees. 

In [21]:
# Importing Relevant Libarys
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate

# Create Model
forest_model = RandomForestRegressor()

forest_model.fit(X_train, y_train)

y_pred = forest_model.predict(X_test)

#Inital Evalulation
print('R-squared:', r2_score(y_test, y_pred).round(3))
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred).round(3))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred).round(3))

R-squared: 0.865
Mean Absolute Error: 2416.93
Mean Squared Error: 20909353.597


In [22]:
# Cross-validation
cv_results = cross_validate(forest_model, X_train, y_train, cv=5, return_train_score=True)

print("Test scores:", cv_results['test_score'].round(3))
print("Training scores:", cv_results['train_score'].round(3))
print("Mean test score:", cv_results['test_score'].mean().round(3))
print("Mean training score:", cv_results['train_score'].mean().round(3))
print("Mean Score Difference:", (cv_results['train_score'].mean()-cv_results['test_score'].mean()).round(3))
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred).round(3))

Test scores: [0.819 0.9   0.794 0.772 0.831]
Training scores: [0.976 0.972 0.976 0.977 0.974]
Mean test score: 0.823
Mean training score: 0.975
Mean Score Difference: 0.152
Mean Absolute Error: 2416.93


<p style="font-size:18px;"> The model is definitely an improvement over the linear models and the accuracy is slowly climbing higher. To ensure the models accuracy I cross validated the data to ensure more accurate scores. However I noticed that the training scores are significantly higher than the testing/validation scores. This suggests that the model is overfitting. I will look to reduce this in the next stage as I believe that the random forest model could be optimized to increase accuracy and create a better model.
    
<p style="font-size:18px;"> To optimize the random forest I will be using a grid search to help determine the best parameters which gives the model the most accuracy.
    
<p style="font-size:18px;"> The model was trained several times to optimize the parameters and produce the best results. The best parameters and values I found are in the code block below:

In [23]:
from sklearn.model_selection import GridSearchCV

#Creating the model
forest_model = RandomForestRegressor()

# Parameters to search
#param_grid = {"n_estimators":[50, 100, 150],
#              "min_samples_split":[5, 10, 15],
#             "min_samples_leaf":[1, 5, 10],
#             "max_depth":[None, 10, 20],}


param_grid = {"n_estimators":[50],
              "min_samples_split":[5],
             "min_samples_leaf":[10],
             "max_depth":[10],}

# Run the model on each parameter combination
grid_search = GridSearchCV(forest_model, param_grid, scoring="r2", return_train_score=True)

grid_search.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'max_depth': [10], 'min_samples_leaf': [10],
                         'min_samples_split': [5], 'n_estimators': [50]},
             return_train_score=True, scoring='r2')

In [24]:
# Print the mean training and validation scores
print("Mean Training Score:", grid_search.cv_results_['mean_train_score'].mean().round(3))
print("Mean Validation Score:", grid_search.cv_results_['mean_test_score'].mean().round(3))
print("Mean Score Difference:", (grid_search.cv_results_['mean_train_score'].mean()-grid_search.cv_results_['mean_test_score'].mean()).round(3))

Mean Training Score: 0.888
Mean Validation Score: 0.846
Mean Score Difference: 0.042


In [25]:
# Best parameters from the grid search
grid_search.best_estimator_

RandomForestRegressor(max_depth=10, min_samples_leaf=10, min_samples_split=5,
                      n_estimators=50)

In [26]:
# Select the model with the best parameters
best_forest = grid_search.best_estimator_

# Make predicitions
y_pred_test = best_forest.predict(X_test)

# Model evaluation
print("Optimized Random Forest Regressor Test Score:", best_forest.score(X_test, y_test).round(5))
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred).round(3))
print("Mean Squared Error:", mean_squared_error(y_test, y_pred_test).round(3))

Optimized Random Forest Regressor Test Score: 0.87909
Mean Absolute Error: 2416.93
Mean Squared Error: 18771220.987


<p style="font-size:18px;"> By optimizing the random forest I have been able to improve the models accuracy which has now made this model the most accurate model thus far. There is also a very minor difference between the test and train score which suggests the overfitting has been improved upon when compared to the default random forest model.
    
<p style="font-size:18px;"> To help visualize the models accuracy I plotted the residuals and the predicted vs actual values on the two graphs below. 

In [27]:
# Calculate residuals
residuals = y_test - y_pred_test.flatten()

# Plot residuals
fig = go.Figure()

# Hover text with predicted values, actual values and residuals
hover_text = [f'Predicted Value: ${pred:.2f}<br>Actual Value: ${actual:.2f}<br>Residual: ${residual:.2f}' 
              for pred, actual, residual in zip(y_pred_test.flatten(), y_test, residuals)]

fig.add_trace(go.Scatter(x=list(range(len(residuals))), y=residuals, mode='markers', 
                         name='Residuals',hoverinfo='text', text=hover_text))

fig.update_layout(title='Residual Plot', xaxis_title='Index', yaxis_title='Residuals')

fig.show()

# Plot predicted vs actuals #################################################################
fig = go.Figure()

hover_text = [f'Predicted Value: ${pred:.2f}<br>Actual Value: ${actual:.2f}' for pred, 
              actual in zip(y_pred_test.flatten(), y_test)]

fig.add_trace(go.Scatter(x=y_test, y=y_pred_test.flatten(), mode='markers', name='Predicted vs Actuals',
                         hoverinfo='text', text=hover_text))

# Add a diagonal line for reference
fig.add_trace(go.Scatter(x=[min(y_test), max(y_test)], y=[min(y_test), max(y_test)], mode='lines', name='Ideal Predictions', 
                         line=dict(color='black', dash='dash')))

fig.update_layout(title='Predicted vs Actuals', xaxis_title='Actuals', yaxis_title='Predicted', showlegend=True)

fig.show()

<p style="font-size:18px;"> This visualization clearly shows that this model is overall more accurate than the linear regression model however the model appears to be consistently predicting higher than the actual values. This model could be further fine tuned in the future to improve accuracy.

# **Gradient Boosting**
    
***

<p style="font-size:18px;"> The next algorithm I will use is gradient boosting which is another ensemble technique that builds models sequentially, with each new model attempting to correct the errors made by the previous ones. It is widely used for both classification and regression tasks.

In [28]:
from sklearn.ensemble import  GradientBoostingRegressor

# Create model
gbr_model = GradientBoostingRegressor()
gbr_model.fit(X_train, y_train)

# Predictions
y_pred = gbr_model.predict(X_test)

#Evaluation
print('R-squared:', r2_score(y_test, y_pred).round(5))
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred).round(3))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred).round(3))

R-squared: 0.87805
Mean Absolute Error: 2446.926
Mean Squared Error: 18932125.408


In [29]:
# Cross-validation
cv_results = cross_validate(gbr_model, X_train, y_train, scoring=['r2'], return_train_score=True)

print("Train - R-squared scores:", cv_results['train_r2'].round(3))
print("Validation - R-squared scores:", cv_results['test_r2'].round(3))
print("Train - Average R-squared score:", cv_results['train_r2'].mean().round(5))
print("Validation - Average R-squared score:", cv_results['test_r2'].mean().round(5))
print("Mean Score Difference:", (cv_results['train_r2'].mean()-cv_results['test_r2'].mean()).round(3))

Train - R-squared scores: [0.913 0.894 0.91  0.92  0.906]
Validation - R-squared scores: [0.832 0.921 0.816 0.791 0.842]
Train - Average R-squared score: 0.90858
Validation - Average R-squared score: 0.84021
Mean Score Difference: 0.068


In [30]:
# Hyperparameters to search

#param_grid = {
#    'n_estimators': [50, 100, 150],
#    'learning_rate': [0.01, 0.1, 0.5],
#    'max_depth': [3, 5, 7]
#}

param_grid = {
    'n_estimators': [50],
    'learning_rate': [0.1],
    'max_depth': [3]
}

# Initialize GridSearchCV with the model and hyperparameter grid
grid_search = GridSearchCV(gbr_model, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit GridSearchCV to the training data
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(),
             param_grid={'learning_rate': [0.1], 'max_depth': [3],
                         'n_estimators': [50]},
             scoring='neg_mean_squared_error')

In [31]:
grid_search.best_params_

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}

In [32]:
# Initialize the model with the best hyperparameters
best_gbr_model = GradientBoostingRegressor(**grid_search.best_params_)

# Train the model on the entire training dataset
best_gbr_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_test = best_gbr_model.predict(X_test)

print("R-squared:", r2_score(y_test, y_pred_test).round(5))
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred).round(3))
print("Mean Squared Error:", mean_squared_error(y_test, y_pred_test).round(3))

R-squared: 0.87908
Mean Absolute Error: 2446.926
Mean Squared Error: 18773256.347


<p style="font-size:18px;"> By optimizing the model I have been able to make a very small improvement to the models accuracy which has now made this model the most accurate model thus far. There is also a very minor difference between the test and train score which suggests the model is not overfitting. Gradient boosting gives a good accuracy just slightly behind the optimized random forest's accuracy.
    
<p style="font-size:18px;"> To help visualize the models accuracy I plotted the residuals and the predicted vs actual values on the two graphs below. 

In [33]:
# Calculate residuals
residuals = y_test - y_pred_test.flatten()

# Plot residuals
fig = go.Figure()

# Hover text with predicted values, actual values and residuals
hover_text = [f'Predicted Value: ${pred:.2f}<br>Actual Value: ${actual:.2f}<br>Residual: ${residual:.2f}' 
              for pred, actual, residual in zip(y_pred_test.flatten(), y_test, residuals)]

fig.add_trace(go.Scatter(x=list(range(len(residuals))), y=residuals, mode='markers', 
                         name='Residuals',hoverinfo='text', text=hover_text))

fig.update_layout(title='Residual Plot', xaxis_title='Index', yaxis_title='Residuals')

fig.show()

# Plot predicted vs actuals #################################################################
fig = go.Figure()

hover_text = [f'Predicted Value: ${pred:.2f}<br>Actual Value: ${actual:.2f}' for pred, 
              actual in zip(y_pred_test.flatten(), y_test)]

fig.add_trace(go.Scatter(x=y_test, y=y_pred_test.flatten(), mode='markers', name='Predicted vs Actuals',
                         hoverinfo='text', text=hover_text))

# Add a diagonal line for reference
fig.add_trace(go.Scatter(x=[min(y_test), max(y_test)], y=[min(y_test), max(y_test)], mode='lines', name='Ideal Predictions', 
                         line=dict(color='black', dash='dash')))

fig.update_layout(title='Predicted vs Actuals', xaxis_title='Actuals', yaxis_title='Predicted', showlegend=True)

fig.show()

<p style="font-size:18px;"> This Visualization clearly shows that this model is overall more accurate than the linear regression model however very much like the random forest the model appears to be consistently predicting higher than the actual values. This model could be further fine tuned in the future to improve accuracy.

# **Neural Network using Tensorflow**
     
***

<p style="font-size:18px;"> The final technique I will try is using a neural network (Tensorflow) to predict a person's medical charges. Neural networks are a set of algorithms inspired by the structure and function of the human brain, designed to recognize patterns and solve complex problems. I will be using TensorFlow which is a popular open-source library developed by Google for implementing and training neural networks. 

In [34]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

# Define dropout rate
dropout_rate = 0.05

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Create The model
nn_model = keras.Sequential([
    layers.Dense(128, activation='relu'),
    layers.Dropout(dropout_rate),
    layers.Dense(64, activation='relu'),
    #layers.Dropout(dropout_rate),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

nn_model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(0.001), metrics=['accuracy'])

history = nn_model.fit(X_train, y_train, epochs=200, batch_size=64, verbose=0, validation_data=(X_test, y_test), callbacks=[early_stopping])

#Make predictions
y_pred = nn_model.predict(X_test)

# Evaluation metrics
print('R² Score:', r2_score(y_test, y_pred).round(5))
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred).round(3))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred).round(3))

2024-06-09 22:04:38.671205: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-09 22:04:38.671349: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-09 22:04:38.838823: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
R² Score: 0.8735
Mean Absolute Error: 1564.416
Mean Squared Error: 19639732.561


In [35]:
# Extracting loss values from the history object
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

# Training &  Validation plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(1, len(training_loss) + 1)), y=training_loss, mode='lines', name='Training Loss'))
fig.add_trace(go.Scatter(x=list(range(1, len(validation_loss) + 1)), y=validation_loss, mode='lines', name='Validation Loss'))

fig.update_layout(title='Training and Validation Loss', xaxis_title='Epochs', yaxis_title='Loss')

fig.show()

<p style="font-size:18px;"> I decided to plot the training and validation loss per epoch to identify if the model is over or under fitting. Overall the model appears to be fitting well to the data.
    
<p style="font-size:18px;"> I trained and tested the model multiple times and saved the best model as "trained_model.h5". Below I evaluate the best model:

In [36]:
# Load the saved model
best_model = tf.keras.models.load_model("/kaggle/input/medical-charges-tensorflow-model/tensorflow2/best/1/trained_model.h5")

# Use the loaded model to make predictions on new data
y_pred_best = best_model.predict(X_test)

# Evaluation metrics
print('R² Score (Loaded Model):', r2_score(y_test, y_pred_best).round(5))
print('Mean Absolute Error (Loaded Model):', mean_absolute_error(y_test, y_pred_best).round(3))
print('Mean Squared Error (Loaded Model):', mean_squared_error(y_test, y_pred_best).round(3))

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
R² Score (Loaded Model): 0.87826
Mean Absolute Error (Loaded Model): 1482.232
Mean Squared Error (Loaded Model): 18900709.82


<p style="font-size:18px;"> The model has a slightly lower R2 score than the other models however has a much better MAE (Mean Absolute Error) which indicates that its predictions are more accurate on average when compared to the other models. Which makes this the best model for predicting a person's medical charges.
    
<p style="font-size:18px;"> To help visualize the models accuracy I plotted the residuals and the predicted vs actual values on the two graphs below.

In [37]:
# Calculate residuals
residuals = y_test - y_pred_best.flatten()

# Plot residuals
fig = go.Figure()

# Hover text with predicted values, actual values and residuals
hover_text = [f'Predicted Value: ${pred:.2f}<br>Actual Value: ${actual:.2f}<br>Residual: ${residual:.2f}' 
              for pred, actual, residual in zip(y_pred_best.flatten(), y_test, residuals)]

fig.add_trace(go.Scatter(x=list(range(len(residuals))), y=residuals, mode='markers', 
                         name='Residuals',hoverinfo='text', text=hover_text))

fig.update_layout(title='Residual Plot', xaxis_title='Index', yaxis_title='Residuals')

fig.show()

# Plot predicted vs actuals #################################################################
fig = go.Figure()

hover_text = [f'Predicted Value: ${pred:.2f}<br>Actual Value: ${actual:.2f}' for pred, 
              actual in zip(y_pred_best.flatten(), y_test)]

fig.add_trace(go.Scatter(x=y_test, y=y_pred_best.flatten(), mode='markers', name='Predicted vs Actuals',
                         hoverinfo='text', text=hover_text))

# Add a diagonal line for reference
fig.add_trace(go.Scatter(x=[min(y_test), max(y_test)], y=[min(y_test), max(y_test)], mode='lines', name='Ideal Predictions', 
                         line=dict(color='black', dash='dash')))

fig.update_layout(title='Predicted vs Actuals', xaxis_title='Actuals', yaxis_title='Predicted', showlegend=True)

fig.show()

<p style="font-size:18px;"> Across the board this model is predicting very close to the actual values with only a few predictions which are significantly inaccurate. Once again this model could be further fine tuned to improve accuracy.

# **Summary**
    
***

<p style="font-size:18px;"> Thank you for making it this far in my notebook!

<p style="font-size:18px;">In this section I will summarize my findings giving an overview of this notebook and explaining the approach I took.

<p style="font-size:18px;">To begin I imported the data and ensured that the data was clean before I began my analysis.

<p style="font-size:18px;">Through my exploration I found that the two factors which affect a person's medical bills the most are:
<ul><li p style="font-size:18px;"> Smoking: If a person smokes their medical bills on average were 3.8x more expensive than a person of the same age who does smoke. 
<li p style="font-size:18px;"> Age: The older a person becomes the more expensive their medical bills as the minimum charges tend to increase per age.</ul>

<p style="font-size:18px;">Unfortunately nothing can be done about aging however this data could be used to show how smoking can increase your medical bills. Suggesting that it is much better for a person not to smoke as their medical bills will likely be significantly lower than their smoking counterpart.

<p style="font-size:18px;">Once I felt that the data had been explored and analyzed to a good degree, I moved onto using machine learning algorithms to see if I could predict a person's medical charges. My findings were as follows:

<p p style="font-size:18px;"> <strong> Linear Regression </strong>
<br>R²: 0.78335
<br>Mean Absolute Error: 4186.509
<br>Mean Squared Error: 33635210.431

<p p style="font-size:18px;"> <strong> Random Forest </strong>
<br>R²: Score: 0.87831
<br>Mean Absolute Error: 2526.348
<br>Mean Squared Error: 18892453.498

<p p style="font-size:18px;"> <strong> Gradient Boosting </strong>
<br>R²: 0.87908
<br>Mean Absolute Error: 2446.926
<br>Mean Squared Error: 18773256.347

<p p style="font-size:18px;"> <strong> Neural Network </strong>
<br>R²: 0.87826
<br>Mean Absolute Error: 1482.232
<br>Mean Squared Error: 18900709.82

<p p style="font-size:18px;"> Overall I found that linear regression was a good starting point however the model is too basic and as a result had a low R² score and high MAE. Both the random forest and gradient boosting models performed very closely with very similar evaluation metrics. The final model was the neural network which had a very similar R² Score when compared to the random forest and gradient boosting models. However the neural network had a significantly lower MAE which makes it the best model for predicting a person's medical charges.

# **Next Steps**
    
***

<p p style="font-size:18px;">Although I have used multiple machine learning algorithms to predict a person medical charges this has barely scrached the surface of machine learning algorithms. There are many more models which I could use and the models I have used could be greatly improved up on. To improve my models I could use the following techniques to name a few:

<p p style="font-size:18px;">
<ul><li style="font-size:18px;"><strong>Feature Engineering</strong> - Create new features from the existing data to provide more relevant and refined information to the model.
<li style="font-size:18px;"><strong>Hyperparameter Tuning</strong> - Use techniques like grid search, random search, and Bayesian optimization to find the best hyperparameters for each model.
<li style="font-size:18px;"><strong>Retraining</strong> - Periodically retrain models with new data to keep them up-to-date and maintain performance.</ul>

<p p style="font-size:18px;"> At this stage I am happy with my findings and insights from the data. In the future I may revisit this notebook to implement the above mentioned techniques to improve upon my models. I hope you enjoyed my notebook and have found my insights interesting!

<p p style="font-size:18px;">Please feel free to give me your feedback!

### **References**

***

Lantz, B. (2013). Machine learning with R : learn how to use R to apply powerful machine learning methods and gain an insight into real-world applications. Packt Publishing.